In [1]:
import numpy as np
from scipy.optimize import linprog, minimize

def f(x):
    return x[0]**2 - x[1]**2

def constraint(x):
    return 2*x[0] + x[1] - 3

def gradient_f(x):
    return np.array([2*x[0], -2*x[1]])

def gradient_constraint(x):
    return np.array([2, 1])

def zoitenjik_method(f, grad_f, constraint, grad_constraint, x0, max_iter=100, epsilon=1e-6):
    x = np.array(x0, dtype=float)
    trajectory = [x.copy()]
    
    for k in range(max_iter):
        # 1. Проверка активного ограничения
        if abs(constraint(x)) > epsilon:
            # Если точка вне ограничения, проектируем на ограничение
            # Решаем задачу минимизации расстояния до ограничения
            c = np.array([1, 1])  # Минимизируем сумму отклонений
            A_eq = grad_constraint(x).reshape(1, -1)
            b_eq = np.array([constraint(x)])
            
            res = linprog(c, A_eq=A_eq, b_eq=b_eq)
            if res.success:
                x = x + res.x
                trajectory.append(x.copy())
                continue
        
        # 2. Вычисляем градиенты
        grad_fx = grad_f(x)
        grad_phix = grad_constraint(x)
        
        # 3. Решаем вспомогательную задачу ЛП
        # Минимизируем grad_f * d при grad_phi * d = 0
        c = grad_fx.flatten()
        A_eq = grad_phix.reshape(1, -1)
        b_eq = np.array([0])
        
        bounds = [(-1, 1), (-1, 1)]  # Ограничения на направление
        res = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds)
        
        if not res.success:
            break
            
        d = res.x
        if np.linalg.norm(d) < epsilon:
            break
            
        # 4. Линейный поиск вдоль направления d
        alpha = 1.0
        rho = 0.5  # Коэффициент уменьшения шага
        c = 0.1    # Параметр достаточного убывания
        
        while True:
            x_new = x + alpha * d
            # Проверка допустимости и условия убывания
            if (abs(constraint(x_new)) < epsilon and 
                f(x_new) < f(x) + c*alpha*np.dot(grad_fx.flatten(), d)):
                break
            alpha *= rho
            if alpha < 1e-10:
                break
                
        x = x + alpha * d
        trajectory.append(x.copy())
        
        if np.linalg.norm(alpha * d) < epsilon:
            break
            
    return x, trajectory

# Начальная точка (должна удовлетворять ограничению)
x0 = np.array([1.0, 1.0])  # 2*1 + 1 - 3 = 0 - удовлетворяет ограничению

# Запуск метода
solution, trajectory = zoitenjik_method(f, gradient_f, constraint, gradient_constraint, x0)

# Вывод результатов
print("Результаты метода Зойтендейка:")
print(f"Найденная точка: ({solution[0]:.6f}, {solution[1]:.6f})")
print(f"Значение функции: {f(solution):.6f}")
print(f"Проверка ограничения: {constraint(solution):.6f}")
print(f"Количество итераций: {len(trajectory)-1}")

Результаты метода Зойтендейка:
Найденная точка: (-49.000000, 101.000000)
Значение функции: -7800.000000
Проверка ограничения: 0.000000
Количество итераций: 100


In [20]:
def f(x):
    return x[0]**2 - x[1]**2

def constraint(x):
    return 2*x[0] + x[1] - 3

def grad_f(x):
    return np.array([2*x[0], -2*x[1]])

def grad_constraint(x):
    return np.array([2, 1])

def feasible_direction_method(f, grad_f, constraint, grad_constraint, x0, max_iter=100, epsilon=1e-6):
    x = np.array(x0, dtype=float)
    trajectory = [x.copy()]
    
    for k in range(max_iter):
        # 1. Проверка активности ограничения
        if abs(constraint(x)) > epsilon:
            # Проекция на допустимое множество
            cons = {'type': 'eq', 'fun': constraint}
            res = minimize(lambda z: np.sum((z - x)**2), x, constraints=cons)
            if res.success:
                x = res.x
                trajectory.append(x.copy())
        
        # 2. Вычисление градиентов
        grad_fx = grad_f(x)
        grad_phix = grad_constraint(x)
        
        # 3. Поиск возможного направления
        # Решаем задачу: min grad_f^T*d при grad_phi^T*d = 0 и ||d|| <= 1
        c = grad_fx
        A_eq = grad_phix.reshape(1, -1)
        b_eq = np.array([0])
        
        # Решаем задачу ЛП для нахождения направления
        res = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=[(-1, 1), (-1, 1)])
        
        if not res.success:
            break
            
        d = res.x
        if np.linalg.norm(d) < epsilon:
            break
            
        # 4. Определение длины шага
        # Минимизируем f(x + αd) при условии constraint(x + αd) = 0
        def phi(alpha):
            return f(x + alpha*d)
            
        # Находим α, при котором удовлетворяется ограничение
        cons = {'type': 'eq', 'fun': lambda a: constraint(x + a*d)}
        alpha_max = 1.0  # Максимальный шаг
        alpha_result = minimize_scalar(phi, bounds=(0, alpha_max), method='bounded')
        
        if not alpha_result.success:
            break
            
        alpha = alpha_result.x
        x_new = x + alpha*d
        
        # 5. Проверка сходимости
        if np.linalg.norm(x_new - x) < epsilon:
            break
            
        x = x_new
        trajectory.append(x.copy())
    
    return x, trajectory

# Начальная точка (должна удовлетворять ограничению)
x0 = np.array([1.0, 1.0])  # 2*1 + 1 = 3 → удовлетворяет

# Запуск метода
solution, trajectory = feasible_direction_method(f, grad_f, constraint, grad_constraint, x0)

# Вывод результатов
print("Результаты метода возможных направлений:")
print(f"Начальная точка: ({x0[0]:.1f}, {x0[1]:.1f})")
print(f"Найденная точка: ({solution[0]:.6f}, {solution[1]:.6f})")
print(f"Значение функции: {f(solution):.6f}")
print(f"Проверка ограничения: {constraint(solution):.6f}")
print(f"Количество итераций: {len(trajectory)-1}")

Результаты метода возможных направлений:
Начальная точка: (1.0, 1.0)
Найденная точка: (-48.999702, 100.999404)
Значение функции: -7799.908799
Проверка ограничения: -0.000000
Количество итераций: 100
